In [49]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [50]:
from secret import Dagshub_PASSWORD

In [51]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="GyanPrakashKushwaha"
os.environ["MLFLOW_TRACKING_PASSWORD"]= Dagshub_PASSWORD

## Entity

In [52]:
from dataclasses import dataclass
from pathlib import Path
import pickle

In [53]:
@dataclass(frozen=True)
class MLFlowTrackingConfig:
    mflow_dir : Path
    test_data: Path
    model_obj : str
    metrics_file: str
    params : dict
    mlflow_uri : str
    target_col : str
    confusion_metrics:str

    

## configuration 

In [54]:
from churnPredictor.utils import *
from churnPredictor.constants import *

In [55]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH):

        self.config = read_yaml(config_file_path)
        self.schema = read_yaml(schema_file_path)
        self.params = read_yaml(params_file_path)

        create_dirs([self.config.artifacts_root])

    def get_mlflow_tracking_config(self) -> MLFlowTrackingConfig:
        config = self.config.mlflow_tracking
        params = self.params.RandomForest
        schema = self.schema.DEPENDET_FEATURES

        # create_dirs([self.config.mflow_dir])

        return MLFlowTrackingConfig(
            mflow_dir=config.mlflow_dir,
            test_data=config.test_data,
            model_obj=config.model_obj_path,
            metrics_file=config.metrics_file_name,
            params=params,
            target_col=schema.Churn,
            mlflow_uri='https://dagshub.com/GyanPrakashKushwaha/Customer-Churn-Prediction.mlflow',
            confusion_metrics=config.confusion_metrics
        )

        

## component

In [56]:
from churnPredictor.components.model_trainer import ModelTrainer
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, precision_score, classification_report
import pandas as pd
import joblib
import mlflow
from urllib.parse import urlparse
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [57]:

class TrackModelPerformance:
    def __init__(self,config:MLFlowTrackingConfig):
        self.config = config

    def evaluate(self,true,pred):
        
        cm = confusion_matrix(true, pred)
        sns.heatmap(data=cm,annot=True, fmt='d', cmap='Blues')
        plt.savefig(self.config.confusion_metrics)
        accuracy = accuracy_score(true, pred)
        recall = recall_score(true, pred)
        precision = precision_score(true, pred)
        report = classification_report(true, pred)

        evaluation_report = {
                    'accuracy': accuracy,
                    'recall': recall,
                    'precision': precision,
                    # 'classification_report': report
                    }
        
        logger.info(f'evaluation_report -> {evaluation_report}')
        return evaluation_report 
    

    def start_mlflow(self):
        try:
            test_data = pd.read_csv(self.config.test_data)
            
            model = pickle.load(open(self.config.model_obj,'rb'))
            logger.info(f'{model} loaded')
            X_test = test_data.drop('Churn',axis=1)
            y_test = test_data['Churn']

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            with mlflow.start_run():
                y_pred = model.predict(X_test)
                evaluation_report = self.evaluate(true=y_test,pred=y_pred)
                with open(self.config.metrics_file, 'w') as json_file:
                    json.dump(evaluation_report, json_file)
                
                mlflow.log_params(self.config.params)

                for metric in evaluation_report:
                    mlflow.log_metric(metric,evaluation_report[metric])
                                
            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, 'model', registered_model_name="random forest")
            else:
                mlflow.sklearn.log_model(model, 'model', registered_model_name="random forest")
        except Exception as e:
            raise CustomException(e)


In [58]:
try:
    config = ConfigurationManager()
    mlflow_tracking_config = config.get_mlflow_tracking_config()
    track_model = TrackModelPerformance(config=mlflow_tracking_config)
    track_model.start_mlflow()
except Exception as e:
    raise CustomException(e)

[2023-09-16 11:29:31,892: INFO: utils: yaml file: config\config.yaml loaded successfully]
[2023-09-16 11:29:31,899: INFO: utils: yaml file: schema.yaml loaded successfully]
[2023-09-16 11:29:31,905: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-09-16 11:29:31,908: INFO: utils: Created artifacts]


[2023-09-16 11:29:33,639: INFO: 2870689772: RandomForestClassifier(n_estimators=132, oob_score=True) loaded]


CustomException: Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_18760\929068938.py] line number [5] error message [Error occured in python script name [C:\Users\Gyanprakash\AppData\Local\Temp\ipykernel_18760\2870689772.py] line number [38] error message [Run with UUID e3b4215491ac4cd8bc0dbfc856ef3c17 is already active. To start a new run, first end the current run with mlflow.end_run(). To start a nested run, call start_run with nested=True]]

## Old code.
```Python

class TrackModelPerformance:
    def __init__(self,config:MLFlowTrackingConfig):
        self.config = config

    def evaluate(self,true,pred):
        
        cm = confusion_matrix(true, pred)
        sns.heatmap(data=cm,annot=True, fmt='d', cmap='Blues')
        plt.savefig(self.config.confusion_metrics)
        accuracy = accuracy_score(true, pred)
        recall = recall_score(true, pred)
        precision = precision_score(true, pred)
        report = classification_report(true, pred)

        evaluation_report = {
                    'accuracy': accuracy,
                    'recall': recall,
                    'precision': precision,
                    # 'classification_report': report
                    }
        
        logger.info(f'evaluation_report -> {evaluation_report}')
        return evaluation_report 
    

    def create_experiment(self,experiment_name,run_name,model,metrics,confusion_matrix=None,params=None):
        try:            
            # mlflow.
            mlflow.set_registry_uri(self.config.mlflow_uri)
            self.tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            mlflow.set_experiment(experiment_name=experiment_name)
            with mlflow.start_run():

                if not params == None:
                    for i in params:
                        mlflow.log_param(i,params[i])

                for metric in metrics:
                    mlflow.log_metric(metric,metrics[metric])


                if not confusion_matrix == None:
                    mlflow.log_artifact(confusion_matrix,'confusion_matrix')
                
                mlflow.log_param('tree',params)
                mlflow.sklearn.log_model(model,self.config.model_obj)
                
                mlflow.set_tag("tag1", "Random Forest")
                mlflow.set_tags({"tag2":"basic model", "tag3":"experimentation"})

                logger.info('Run - %s is logged to Experiment - %s' %(run_name, experiment_name))
        except Exception as e:
            raise CustomException(e)


    def start_mlflow(self):
        try:
            test_data = pd.read_csv(self.config.test_data)
            
            model = pickle.load(open(self.config.model_obj,'rb'))
            logger.info(f'{model} loaded')
            X_test = test_data.drop('Churn',axis=1)
            y_test = test_data['Churn']

            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            y_pred = model.predict(X_test)
            evaluation_report = self.evaluate(true=y_test,pred=y_pred)
            with open(self.config.metrics_file, 'w') as json_file:
                json.dump(evaluation_report, json_file)

            
            self.create_experiment(experiment_name='Random-Forest-clf',
                                run_name='experiment_1',
                                model=model,
                                metrics=evaluation_report,
                                params=self.config.params,
                                confusion_matrix=self.config.confusion_metrics)

            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, self.config.model_obj, registered_model_name="random forest")
            else:
                mlflow.sklearn.log_model(model,  self.config.model_obj, registered_model_name="random forest")
        except Exception as e:
            raise CustomException(e)


```